In [2]:
import json
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_one = []
# data_two = []
# data_three = []
# data_four = []
reviewdata_df = []

In [4]:
data_file = open('F:\yelp\yelp_academic_dataset_review.json')
length = 2225213  # I calculated length prior to this calculation
for i, line in enumerate(data_file):
    if i<(length/4):
        data_one.append(json.loads(line))
data_file.close()

In [5]:
df = pd.DataFrame(data_one)

In [6]:
df.shape

(556303, 8)

In [7]:
data_main = []
with open('F:\yelp\yelp_academic_dataset_user.json') as f:
    for line in f:
        data_main.append(json.loads(line))

In [8]:
df_user = pd.DataFrame(data_main)

In [9]:
def user_with_friends(x):
    if len(x['friends']) != 0:
        return x

In [10]:
df_user_with_friends = df_user.apply(user_with_friends, axis=1)

In [11]:
# if len(yo_df['friends']) == 0:
#     print "yo"
df_user_with_friends = df_user_with_friends.dropna()


In [12]:
df_user.shape


(552339, 11)

In [13]:
df_user_with_friends.shape

(249440, 11)

In [14]:
merged_df = pd.merge(df, df_user_with_friends, on='user_id', how='inner')

In [15]:
merged_df.head(n=1)

,business_id,date,review_id,stars,text,type_x,user_id,votes_x,average_stars,compliments,elite,fans,friends,name,review_count,type_y,votes_y,yelping_since
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,{},[],2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",2013-05


In [16]:
merged_df.shape

(394815, 18)

In [17]:
merged_df.stars.value_counts(ascending=True)

2     36085
1     38347
3     63586
4    124241
5    132556
Name: stars, dtype: int64

In [18]:
data_business = []
with open('F:\yelp\yelp_academic_dataset_business.json') as f:
    for line in f:
        data_business.append(json.loads(line))

In [19]:
df_business = pd.DataFrame(data_business)

In [20]:
business_merged_df = pd.merge(merged_df, df_business, on='business_id', how='inner')

In [21]:
business_merged_df.shape

(394815, 32)

In [22]:
business_merged_df.drop(['neighborhoods', 'latitude', 'longitude', 'elite', 'yelping_since', 'compliments', 'full_address', 'city', 'state', 'categories', 'open', 'hours', 'attributes'], axis=1, inplace=True)

In [23]:
business_merged_df.head(n=1)

,business_id,date,review_id,stars_x,text,type_x,user_id,votes_x,average_stars,fans,friends,name_x,review_count_x,type_y,votes_y,name_y,review_count_y,stars_y,type
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",Clancy's Pub,4,3.5,business


In [24]:
for index, row in business_merged_df.head(n=5).iterrows():
    if len(row['friends']) != 0:
        print index

0
1
2
3
4


In [25]:
for index, row in business_merged_df.head(n=5).iterrows():
    print index

0
1
2
3
4


In [26]:
%timeit business_sorted_df = business_merged_df.sort_index(by=['business_id'], ascending=[True])

1 loops, best of 3: 1.85 s per loop


F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:257: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [27]:
#1 is b-id, 3 is review id, 4 is stars_x, 7 is user id, 11 friend ids, 18 is business rating
for row in business_merged_df.head(n=1).itertuples():
    print "The business: " + str(row[1]) + " has an avg rating of: " + str(row[18])
    print "User " + str(row[7]) +" rated " + str(row[1]) + " with " + str(row[4]) + " stars. "
    print "Friends: " + str(row[11])
    for fid in row[11]:
        print fid

The business: UsFtqoBl7naz8AVUBZMjQQ has an avg rating of: 3.5
User uK8tzraOp4M5u3uYrqIBXg rated UsFtqoBl7naz8AVUBZMjQQ with 5 stars. 
Friends: [u'53wSMIztehsjP-vJZBS54A', u'CXhOU5apSRSTtEUiyoqH3Q']
53wSMIztehsjP-vJZBS54A
CXhOU5apSRSTtEUiyoqH3Q


In [ ]:
#####Under construction
for row in business_merged_df.head(n=1).itertuples():
    print "The business: " + str(row[1]) + " has an avg rating of: " + str(row[18])
    print "User " + str(row[7]) +" rated " + str(row[1]) + " with " + str(row[4]) + " stars. "
    count = 0
    friends = 0
    friends_who_rated = 0
    for fid in row[11]:
        friends += 1

In [1]:
new_df = pd.DataFrame()
for row in business_merged_df.head(n=10).itertuples():
    if len(row['friends']) != 0:
        business_rating = row['stars_y']
        business_id = row['business_id']
        user_id = row['user_id']
        print "The business: " + str(business_id) + " has an avg rating of: " + str(business_rating)  
        user_rating = row['stars_x']
        print "User " + str(row['user_id']) +" rated " + str(business_id) + " with " + str(user_rating) + " stars. "
        count = 0
        friends = 0
        friends_who_rated = 0
        for fid in row['friends']:
#             if row['user_id'] == fid:
            friends += 1
            if any(business_merged_df[business_merged_df['user_id'] == fid]):
                for idx in business_merged_df[business_merged_df['user_id'] == fid].index:
#                    print business_merged_df.ix[idx]['user_id'] + "  =  " + fid   ##prints dat at idx, user & friend id r same
#                    print str(business_merged_df.ix[idx]['business_id']) + " at index "
                    if business_merged_df.ix[idx]['business_id'] == business_id:
                        #print "Friend " + business_merged_df.ix[idx]['user_id'] + " rated " + business_merged_df.ix[idx]['business_id']
                        #print "these many stars: " + business_merged_df.ix[idx]['stars_x'].astype(str)
                        count += business_merged_df.ix[idx]['stars_x']
                        friends_who_rated += 1
        #print "Sum of total ratings given by friends: " + str(count)
        #print "Total friends: " + str(friends)
        #print "Total friends who rated the business: " + str(friends_who_rated)
        if friends_who_rated != 0:
            average = float(count/float(friends_who_rated))
            #print "Friends average: " + str(average)
            data = pd.DataFrame({"business_id": [business_id], "business_stars": [business_rating], "user_stars": [user_rating], 
                                 "friends_avg_rating": [average], "user_id": [user_id], "friends_who_rated": [friends_who_rated],
                                 "total_friends": [friends]})
            new_df = new_df.append(data, ignore_index=True)
#                     else:
#                         print "no"
#             if any(business_merged_df['user_id'] == fid):
#                 print fid + " = " + business_merged_df['user_id']
#                 count += row['stars_x']
#         print "total friends' votes: %s" % count
#         print "total friends: %s" % friends
#                 count += row['stars_x']
#             print count


NameError: name 'pd' is not defined

In [28]:
merged_df.head(n=1)

,business_id,date,review_id,stars,text,type_x,user_id,votes_x,average_stars,compliments,elite,fans,friends,name,review_count,type_y,votes_y,yelping_since
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,{},[],2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",2013-05


In [28]:
def look_for_user_ratings(df, value):
    ur = []
    for row in df.itertuples():
        if value in row[1:]:
            ur.append({
                    "user_id": row[7],
                    "user_rating": row[4],
                })
    return ur

look_for_user_ratings(df, "5UmKMjUEUNdYWqANhGckJw")

[{'user_id': u'PUFPaY9KxDAcGqfsorJp3Q', 'user_rating': 4},
 {'user_id': u'Iu6AxdBYGR4A0wspR9BYHA', 'user_rating': 5},
 {'user_id': u'auESFwWvW42h6alXgFxAXQ', 'user_rating': 5}]

In [32]:
business_user = {}

for row in business_merged_df.head(n=6000).itertuples():
    business_id = row[1]
    #print look_for_user_ratings(df, business_id)
    business_user.update({
        business_id: look_for_user_ratings(business_merged_df.head(n=6000), business_id)
    })


F:\PythonIDE\lib\site-packages\ipykernel\__main__.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [33]:
print business_user

{u'q_2HooHFCE9wAwmXQPkuyg': [{'user_rating': 5, 'user_id': u'fhNxoMwwTipzjO8A9LFe8Q'}, {'user_rating': 3, 'user_id': u'8fApIAMHn2MZJFUiCQto5Q'}, {'user_rating': 3, 'user_id': u'tAKjY3bQXH51msJbOHYPmQ'}, {'user_rating': 5, 'user_id': u'IbvOxKSps_K5wa3a2_jc-Q'}, {'user_rating': 3, 'user_id': u'7DeM4C66zVflYNrZn_Vv9g'}, {'user_rating': 4, 'user_id': u'WMTm9HHRA3EewoxTX1Gleg'}, {'user_rating': 5, 'user_id': u'WMTm9HHRA3EewoxTX1Gleg'}, {'user_rating': 4, 'user_id': u'o9XGClSBzCsQxbcCzTPdnQ'}, {'user_rating': 2, 'user_id': u'_vS9qNQvx8wVUOIkj4qfAg'}, {'user_rating': 4, 'user_id': u'LYKMElh2u27sPIzoQFaGIQ'}, {'user_rating': 4, 'user_id': u'8hWBnQA_ckmXoRHYPUmm3g'}, {'user_rating': 2, 'user_id': u'LhgQq1x4n9ardg1PFo8vgg'}, {'user_rating': 4, 'user_id': u'FyCBkNXwoI_6X6apbslg4g'}, {'user_rating': 5, 'user_id': u'FyCBkNXwoI_6X6apbslg4g'}, {'user_rating': 4, 'user_id': u'G7KQF0U0p25nf07dObPeVg'}, {'user_rating': 3, 'user_id': u'N_cH3QA_eXXmnzz-nzEeKA'}, {'user_rating': 3, 'user_id': u'gtJJBaQwTwE

In [33]:
print business_user.get('q_2HooHFCE9wAwmXQPkuyg')

[{'user_rating': 5, 'user_id': u'fhNxoMwwTipzjO8A9LFe8Q'}, {'user_rating': 3, 'user_id': u'8fApIAMHn2MZJFUiCQto5Q'}, {'user_rating': 3, 'user_id': u'tAKjY3bQXH51msJbOHYPmQ'}, {'user_rating': 5, 'user_id': u'IbvOxKSps_K5wa3a2_jc-Q'}, {'user_rating': 3, 'user_id': u'7DeM4C66zVflYNrZn_Vv9g'}, {'user_rating': 4, 'user_id': u'WMTm9HHRA3EewoxTX1Gleg'}, {'user_rating': 5, 'user_id': u'WMTm9HHRA3EewoxTX1Gleg'}, {'user_rating': 4, 'user_id': u'o9XGClSBzCsQxbcCzTPdnQ'}, {'user_rating': 2, 'user_id': u'_vS9qNQvx8wVUOIkj4qfAg'}, {'user_rating': 4, 'user_id': u'LYKMElh2u27sPIzoQFaGIQ'}, {'user_rating': 4, 'user_id': u'8hWBnQA_ckmXoRHYPUmm3g'}, {'user_rating': 2, 'user_id': u'LhgQq1x4n9ardg1PFo8vgg'}, {'user_rating': 4, 'user_id': u'FyCBkNXwoI_6X6apbslg4g'}, {'user_rating': 5, 'user_id': u'FyCBkNXwoI_6X6apbslg4g'}, {'user_rating': 4, 'user_id': u'G7KQF0U0p25nf07dObPeVg'}, {'user_rating': 3, 'user_id': u'N_cH3QA_eXXmnzz-nzEeKA'}, {'user_rating': 3, 'user_id': u'gtJJBaQwTwE45NnNF0coVA'}, {'user_rating

In [34]:
for row in business_merged_df.head(n=6000).itertuples():
    if row[1] == "q_2HooHFCE9wAwmXQPkuyg":
        print "user id" + str(row[7]) + " rated " + str(row[4])

user idfhNxoMwwTipzjO8A9LFe8Q rated 5
user id8fApIAMHn2MZJFUiCQto5Q rated 3
user idtAKjY3bQXH51msJbOHYPmQ rated 3
user idIbvOxKSps_K5wa3a2_jc-Q rated 5
user id7DeM4C66zVflYNrZn_Vv9g rated 3
user idWMTm9HHRA3EewoxTX1Gleg rated 4
user idWMTm9HHRA3EewoxTX1Gleg rated 5
user ido9XGClSBzCsQxbcCzTPdnQ rated 4
user id_vS9qNQvx8wVUOIkj4qfAg rated 2
user idLYKMElh2u27sPIzoQFaGIQ rated 4
user id8hWBnQA_ckmXoRHYPUmm3g rated 4
user idLhgQq1x4n9ardg1PFo8vgg rated 2
user idFyCBkNXwoI_6X6apbslg4g rated 4
user idFyCBkNXwoI_6X6apbslg4g rated 5
user idG7KQF0U0p25nf07dObPeVg rated 4
user idN_cH3QA_eXXmnzz-nzEeKA rated 3
user idgtJJBaQwTwE45NnNF0coVA rated 3
user id_7Lg8iTWpFc0O7PEA8FmbQ rated 4
user idSo32N7bSbUd1RwhFtI6jTQ rated 4
user idjwKgiDjVEC8DPjG5NTGllg rated 2
user idoSo_3ARkzud3Wjc7evC2tg rated 3
user idljGxYz9as2xIvojzh-5Row rated 1
user idE62SZVU_FsbCYaVnMRs4TA rated 3
user idroReLynNBIuqwH47ow77SQ rated 5
user idiTmWHtltCtk0Gm55AOxrUA rated 3
user idiTmWHtltCtk0Gm55AOxrUA rated 3
user idiTmWH

In [35]:
df_user_with_friends.head(n=1)

,average_stars,compliments,elite,fans,friends,name,review_count,type,user_id,votes,yelping_since
0,4.14,"{u'profile': 8, u'cute': 15, u'funny': 11, u'p...","[2005, 2006]",70,"[rpOyqD_893cqmDAtJLbdog, 4U9kSBLuBDU391x6bxU-Y...",Russel,108,user,18kPq7GPye-YQ3LyKyAZPw,"{u'funny': 167, u'useful': 280, u'cool': 245}",2004-10


In [36]:
##This is the 2nd hashmap

user_friends = {}

for row in df_user_with_friends.itertuples():
    user_id = row[9]
#    print user_id
    user_friends.update({
        user_id: row[5],
    })

In [70]:
print user_friends

{u'rpOyqD_893cqmDAtJLbdog': [u'18kPq7GPye-YQ3LyKyAZPw', u'4U9kSBLuBDU391x6bxU-YA', u'fHtTaujcyKvXglE33Z5yIw', u'SIBCL7HBkrP4llolm4SC2A', u'8J4IIYcqBlFch8T90N923A', u'ysYmC-ufbdmVEX9yAv-VEQ', u'UTS9XcT14H2ZscRIf0MYHQ', u'1blidZhgxDVSBuJ_dbTU6g', u'HDQixQ-WZEV0LVPJlIGQeQ', u'91ovYNfGfPiqe6GbTcPwTg', u'8VJUJbrfPYJC0vMAtXGR_g', u'anTtulQ9G2b3SipCEwGVHg', u'Re447krbp0VQVX3Dk4Tsog', u'XejvgD4CrkR7FeIkjwAjtQ', u'Ym2-9Rdeb_PPher2F6qksQ', u'i63u3SdbrLsP4FxiSKP0Zw', u'T7t5UdadpHua-YnmiFpcZw', u'pnrGw4ciBXJ6U5QB2m0F5g', u'ytjCBxosVSqCOQ62c4KAxg', u'K4FAia2Iy5MVnmBLfS-mCg', u'o-6ZHG2iUO3oooMXQawd9g', u'r5uiIxwJ-I-oHBkNY2Ha3Q', u'niWoSKswEbooJC_M7HMbGw', u'kwoxiKMyoYjB1wTCYAjYRg', u'F4sfffJ8-VCNFLUT0q6Gtg', u'TNJRTBrl0yjtpAACr1Bthg', u'W0M8csYKD2ryUsr6OsACyA', u'HKvXA96cbqAawKCU7xyGSw', u'EoTooNBcxu0Hy0-XeZI8aw', u'9A8OuP6XwLwnNb9ov3_Ncw', u'27MmRg8LfbZXNEHkEnKSdA', u'WsPgO3ycZjRp7wFh18JdMQ', u'Bn4sJUTtKFZQt0FKHF2Adw', u'AC84oF7Ciuw7M52RszgWUA', u'uguXfIEpI65jSCH5MgUDgA', u'6VZNGc2h2Bn-uyuEXgOt5g',

In [41]:
json.dumps(user_friends, open("C:\Users\Shubhanshu\Desktop\yelp\\user_friends_json_dump.txt",'w'))

In [54]:
business_merged_df.head(n=1)

,business_id,date,review_id,stars_x,text,type_x,user_id,votes_x,average_stars,fans,friends,name_x,review_count_x,type_y,votes_y,name_y,review_count_y,stars_y,type
0,UsFtqoBl7naz8AVUBZMjQQ,2013-11-08,Di3exaUCFNw1V4kSNW5pgA,5,All the food is great here. But the best thing...,review,uK8tzraOp4M5u3uYrqIBXg,"{u'funny': 0, u'useful': 0, u'cool': 0}",3.84,2,"[53wSMIztehsjP-vJZBS54A, CXhOU5apSRSTtEUiyoqH3Q]",Robin,46,user,"{u'funny': 2, u'useful': 36, u'cool': 5}",Clancy's Pub,4,3.5,business


In [37]:
def look_for_business_ratings(business_user_dict, business_id, user_ids):
    ratings = business_user_dict.get(business_id)
    sum_of_stars = 0
    count = 0
    business_reviewers = [li['user_id'] for li in ratings]
    for i, uid in enumerate(business_reviewers):
        for user in user_ids:
            if uid == user:
                count += 1
                sum_of_stars += ratings[i]['user_rating']
    avg = float(sum_of_stars/float(count))
    return avg

In [40]:
new_df = []
for row in business_merged_df.head(n=5000).itertuples():
    business_id = row[1]
    user_id = row[7]
    ratings = business_user.get(business_id)
    
    friends = user_friends.get(user_id)
    business_reviewers = [li['user_id'] for li in ratings]

    common_reviewers = list(set(friends) & set(business_reviewers))
    if len(common_reviewers) == 0:
        continue
    else:
        friends_avg = look_for_business_ratings(business_user, business_id, common_reviewers)
        
    new_df.append({
            "Business_id": business_id,
            "User_id": user_id,
            "friends_avg_rating": friends_avg,
            "user_rating": row[4],
            "friends_who_rated": len(common_reviewers),
            "friends": len(friends),
            "no_of_reviewers": len(business_reviewers),
            "business_rating": row[18]
        })

final_df = pd.DataFrame(new_df)

In [41]:
 final_df.head(n=10)

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,mVHrayjG3uZ_RLHkLj-AMg,8fApIAMHn2MZJFUiCQto5Q,5,2,5.000000,1,9,5
1,mVHrayjG3uZ_RLHkLj-AMg,tAKjY3bQXH51msJbOHYPmQ,5,31,5.000000,1,9,5
2,YssP-_6iP0SLBsqUQGs6rQ,nEYPahVwXGD2Pjvgkm7QqQ,4,1444,4.000000,22,59,5
3,YssP-_6iP0SLBsqUQGs6rQ,GYNfjDbSlpdRK2qC8RqatA,4,45,4.000000,3,59,1
4,YssP-_6iP0SLBsqUQGs6rQ,r_96n9O0jNzbNCZDkSB0EQ,4,6,4.000000,1,59,5
5,YssP-_6iP0SLBsqUQGs6rQ,Enzdhh-rV5JgC4lgg0IdFw,4,149,3.333333,3,59,3
6,YssP-_6iP0SLBsqUQGs6rQ,ytgLwKzD6B4af5vW56RpJg,4,2,5.000000,1,59,5
7,YssP-_6iP0SLBsqUQGs6rQ,gtJJBaQwTwE45NnNF0coVA,4,64,4.333333,3,59,5
8,YssP-_6iP0SLBsqUQGs6rQ,vFgVosI6MUPX8Y23hmuMXw,4,5,5.000000,1,59,5
9,YssP-_6iP0SLBsqUQGs6rQ,RxEegiMllenlyhYgJqsTSQ,4,16,5.000000,1,59,4


In [42]:
final_df['friends_avg_rating'].corr()['user_rating']

TypeError: corr() takes at least 2 arguments (1 given)

In [43]:
corr_df = final_df[['friends_avg_rating', 'user_rating']]

In [44]:
corr_value = np.correlate(corr_df["friends_avg_rating"], corr_df["user_rating"])

In [45]:
corr_value

array([ 37738.72770982])

In [46]:
corr_df.head(n=5).corr()

,friends_avg_rating,user_rating
friends_avg_rating,1.000000,0.408248
user_rating,0.408248,1.000000


In [73]:
stars = [1,2,3,4,5]
for star in stars:
    final_df_1 = final_df.head(n=50)[(final_df.head(n=50).user_rating == 1)]
    final_df_1

In [42]:
final_df_1 = final_df[(final_df.business_rating >= 1) & (final_df.business_rating <= 2)]
final_df_2 = final_df[(final_df.user_rating == 2)]
final_df_3 = final_df[(final_df.user_rating == 3)]
final_df_4_5 = final_df[(final_df.business_rating >= 4) & (final_df.business_rating <= 5)]
final_df_4 = final_df[(final_df.business_rating == 4)]
final_df_5 = final_df[(final_df.user_rating == 5)]

In [148]:
final_df_5 = final_df[(final_df.business_rating == 5)]

In [48]:
corr_df_4 = final_df_4[['friends_avg_rating', 'user_rating']]

In [43]:
final_df_4

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
2,YssP-_6iP0SLBsqUQGs6rQ,nEYPahVwXGD2Pjvgkm7QqQ,4,1444,4.000000,22,59,5
3,YssP-_6iP0SLBsqUQGs6rQ,GYNfjDbSlpdRK2qC8RqatA,4,45,4.000000,3,59,1
4,YssP-_6iP0SLBsqUQGs6rQ,r_96n9O0jNzbNCZDkSB0EQ,4,6,4.000000,1,59,5
5,YssP-_6iP0SLBsqUQGs6rQ,Enzdhh-rV5JgC4lgg0IdFw,4,149,3.333333,3,59,3
6,YssP-_6iP0SLBsqUQGs6rQ,ytgLwKzD6B4af5vW56RpJg,4,2,5.000000,1,59,5
7,YssP-_6iP0SLBsqUQGs6rQ,gtJJBaQwTwE45NnNF0coVA,4,64,4.333333,3,59,5
8,YssP-_6iP0SLBsqUQGs6rQ,vFgVosI6MUPX8Y23hmuMXw,4,5,5.000000,1,59,5
9,YssP-_6iP0SLBsqUQGs6rQ,RxEegiMllenlyhYgJqsTSQ,4,16,5.000000,1,59,4
10,YssP-_6iP0SLBsqUQGs6rQ,iTmWHtltCtk0Gm55AOxrUA,4,93,5.000000,2,59,4
11,YssP-_6iP0SLBsqUQGs6rQ,sPQ10u9ZsrCVCKhQylFxgQ,4,89,4.428571,4,59,2


In [44]:
final_df_4 = final_df_4.reset_index()

In [45]:
final_df_4.head(n=10)

,index,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,2,YssP-_6iP0SLBsqUQGs6rQ,nEYPahVwXGD2Pjvgkm7QqQ,4,1444,4.000000,22,59,5
1,3,YssP-_6iP0SLBsqUQGs6rQ,GYNfjDbSlpdRK2qC8RqatA,4,45,4.000000,3,59,1
2,4,YssP-_6iP0SLBsqUQGs6rQ,r_96n9O0jNzbNCZDkSB0EQ,4,6,4.000000,1,59,5
3,5,YssP-_6iP0SLBsqUQGs6rQ,Enzdhh-rV5JgC4lgg0IdFw,4,149,3.333333,3,59,3
4,6,YssP-_6iP0SLBsqUQGs6rQ,ytgLwKzD6B4af5vW56RpJg,4,2,5.000000,1,59,5
5,7,YssP-_6iP0SLBsqUQGs6rQ,gtJJBaQwTwE45NnNF0coVA,4,64,4.333333,3,59,5
6,8,YssP-_6iP0SLBsqUQGs6rQ,vFgVosI6MUPX8Y23hmuMXw,4,5,5.000000,1,59,5
7,9,YssP-_6iP0SLBsqUQGs6rQ,RxEegiMllenlyhYgJqsTSQ,4,16,5.000000,1,59,4
8,10,YssP-_6iP0SLBsqUQGs6rQ,iTmWHtltCtk0Gm55AOxrUA,4,93,5.000000,2,59,4
9,11,YssP-_6iP0SLBsqUQGs6rQ,sPQ10u9ZsrCVCKhQylFxgQ,4,89,4.428571,4,59,2


In [46]:
plt.scatter(final_df_4.user_rating, final_df_4.friends_avg_rating)

In [47]:
plt.show()

F:\PythonIDE\lib\site-packages\matplotlib\collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [89]:
plt.plot([0, 4], [4, 0], 'k-')

In [90]:
plt.show()

In [48]:
#does makes some sense
plt.axhline(y=4)
plt.scatter(final_df_4.user_rating, final_df_4.friends_avg_rating, s=final_df_4.friends_who_rated)

In [49]:
plt.show()

In [140]:
#senseless
plt.axhline(y=4)
plt.scatter(final_df_4.user_rating, final_df_4.business_rating, c='b', label='user & business')
plt.scatter(final_df_4.friends_avg_rating, final_df_4.business_rating, c='r', label='friends_avg & business')

In [141]:
plt.show()

In [50]:
#this is the one making the most sense
plt.axhline(y=4)
plt.scatter(final_df_4.index, final_df_4.user_rating, c='b', label='user ratings')
plt.scatter(final_df_4.index, final_df_4.friends_avg_rating, c='r', label='friends_avg')

In [51]:
plt.show()

In [125]:
final_df_4['user_rating'].head(n=50).plot(style='.')

In [126]:
plt.show()

In [142]:
#not making that much sense
plt.axhline(y=4)
final_df_4['user_rating'].plot(marker='o', color='green')
final_df_4['friends_avg_rating'].plot(marker='o', color='blue')

In [143]:
plt.show()

In [149]:
#this is the one making the most sense
plt.axhline(y=5)
plt.scatter(final_df_5.index, final_df_5.user_rating, c='b', label='user ratings')
plt.scatter(final_df_5.index, final_df_5.friends_avg_rating, c='r', label='friends_avg')

In [150]:
plt.show()

In [151]:
final_df_5

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,mVHrayjG3uZ_RLHkLj-AMg,8fApIAMHn2MZJFUiCQto5Q,5,2,5,1,9,5
1,mVHrayjG3uZ_RLHkLj-AMg,tAKjY3bQXH51msJbOHYPmQ,5,31,5,1,9,5


In [52]:
final_df_3 = final_df[(final_df.business_rating == 3)]

In [54]:
final_df_3.reset_index(drop=True)

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,Ln34kwDUNZPLhVhUqauLWQ,0RNjwIoruU0aQ3yMGiZBdA,3,1,1.000000,1,9,1
1,Ln34kwDUNZPLhVhUqauLWQ,lMZD3OpQOb1VijGm2sbfbQ,3,5,1.000000,1,9,1
2,gClSqcwuLsgyZRCzhJc7Dg,JPPhyFE-UE453zA6K0TVgw,3,31,3.000000,1,7,5
3,gClSqcwuLsgyZRCzhJc7Dg,T_wjLgPOPXry7Bea4MzoVQ,3,79,3.000000,1,7,3
4,gClSqcwuLsgyZRCzhJc7Dg,T_wjLgPOPXry7Bea4MzoVQ,3,79,3.000000,1,7,1
5,gClSqcwuLsgyZRCzhJc7Dg,Q3fFv_ft17OyV-NRF1iQxw,3,59,3.000000,1,7,3
6,gClSqcwuLsgyZRCzhJc7Dg,UD7Y1CqfY6mDmRwIuCf6nA,3,955,3.200000,4,7,3
7,gClSqcwuLsgyZRCzhJc7Dg,nYKao-8w_IOj4Mv8QWb2vg,3,30,3.000000,1,7,4
8,eznh0XjVtv924K-KYx_ilQ,JPPhyFE-UE453zA6K0TVgw,3,31,3.333333,2,31,3
9,eznh0XjVtv924K-KYx_ilQ,bvu13GyOUwhEjPum2xjiqQ,3,230,3.454545,7,31,3


In [55]:
#this is the one making the most sense
plt.axhline(y=3)
plt.scatter(final_df_3.index, final_df_3.user_rating, c='b', label='user ratings')
plt.scatter(final_df_3.index, final_df_3.friends_avg_rating, c='r', label='friends_avg')

In [56]:
plt.show()

In [60]:
final_df

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,mVHrayjG3uZ_RLHkLj-AMg,8fApIAMHn2MZJFUiCQto5Q,5.0,2,5.000000,1,9,5
1,mVHrayjG3uZ_RLHkLj-AMg,tAKjY3bQXH51msJbOHYPmQ,5.0,31,5.000000,1,9,5
2,YssP-_6iP0SLBsqUQGs6rQ,nEYPahVwXGD2Pjvgkm7QqQ,4.0,1444,4.000000,22,59,5
3,YssP-_6iP0SLBsqUQGs6rQ,GYNfjDbSlpdRK2qC8RqatA,4.0,45,4.000000,3,59,1
4,YssP-_6iP0SLBsqUQGs6rQ,r_96n9O0jNzbNCZDkSB0EQ,4.0,6,4.000000,1,59,5
5,YssP-_6iP0SLBsqUQGs6rQ,Enzdhh-rV5JgC4lgg0IdFw,4.0,149,3.333333,3,59,3
6,YssP-_6iP0SLBsqUQGs6rQ,ytgLwKzD6B4af5vW56RpJg,4.0,2,5.000000,1,59,5
7,YssP-_6iP0SLBsqUQGs6rQ,gtJJBaQwTwE45NnNF0coVA,4.0,64,4.333333,3,59,5
8,YssP-_6iP0SLBsqUQGs6rQ,vFgVosI6MUPX8Y23hmuMXw,4.0,5,5.000000,1,59,5
9,YssP-_6iP0SLBsqUQGs6rQ,RxEegiMllenlyhYgJqsTSQ,4.0,16,5.000000,1,59,4


In [57]:
final_df_1 = final_df[(final_df.business_rating == 1)]

In [58]:
final_df_1.shape

(0, 8)

In [59]:
final_df.business_rating.value_counts(ascending=True)

5.0       2
3.0      90
2.5     101
4.5     473
3.5     885
4.0    1054
Name: business_rating, dtype: int64

In [61]:
final_df_3_point_5 = final_df[(final_df.business_rating == 3.5)]

In [62]:
final_df_3_point_5.shape

(885, 8)

In [63]:
final_df_3_point_5.reset_index(drop=True)

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,1qCuOcks5HRv67OHovAVpg,I_47G-R2_egp7ME5u_ltew,3.5,6,5.000000,1,17,1
1,1qCuOcks5HRv67OHovAVpg,I_47G-R2_egp7ME5u_ltew,3.5,6,5.000000,1,17,3
2,1qCuOcks5HRv67OHovAVpg,JPPhyFE-UE453zA6K0TVgw,3.5,31,4.000000,1,17,4
3,1qCuOcks5HRv67OHovAVpg,_vS9qNQvx8wVUOIkj4qfAg,3.5,67,5.000000,1,17,3
4,1qCuOcks5HRv67OHovAVpg,4DrkZqFqLcyRcLWuQUXFeQ,3.5,410,2.750000,3,17,5
5,1qCuOcks5HRv67OHovAVpg,KSS6v5fcZNoelu4eEZCFcw,3.5,33,2.000000,1,17,4
6,1qCuOcks5HRv67OHovAVpg,J_q8jN623uew8ZzMChXBCA,3.5,117,4.000000,1,17,2
7,1qCuOcks5HRv67OHovAVpg,Xez2YmtfrixCNZe2CagFRw,3.5,14,5.000000,1,17,4
8,1qCuOcks5HRv67OHovAVpg,2BYGvFAgV87LgIk4EqLtwg,3.5,10,4.000000,1,17,4
9,bwu0cUZfksuu-uzTPrrl2A,I_47G-R2_egp7ME5u_ltew,3.5,6,3.500000,2,424,3


In [64]:
#this is the one making the most sense
plt.axhline(y=3.5)
plt.scatter(final_df_3_point_5.index, final_df_3_point_5.user_rating, c='b', label='user ratings')
plt.scatter(final_df_3_point_5.index, final_df_3_point_5.friends_avg_rating, c='r', label='friends_avg')

In [65]:
plt.show()

In [66]:
final_df_4_point_5 = final_df[(final_df.business_rating == 4.5)]

In [67]:
final_df_4_point_5.shape

(473, 8)

In [68]:
final_df_4_point_5.reset_index(drop=True)

,Business_id,User_id,business_rating,friends,friends_avg_rating,friends_who_rated,no_of_reviewers,user_rating
0,2X5G4Ujq0s4Wfn4TC7gX0g,JPPhyFE-UE453zA6K0TVgw,4.5,31,4.400000,5,193,4
1,2X5G4Ujq0s4Wfn4TC7gX0g,8fApIAMHn2MZJFUiCQto5Q,4.5,2,5.000000,1,193,4
2,2X5G4Ujq0s4Wfn4TC7gX0g,q7MrNVt1FE23rwtWmPYWHg,4.5,27,4.333333,3,193,4
3,2X5G4Ujq0s4Wfn4TC7gX0g,9cCTmiJ7hz35rHIdr8n9kA,4.5,997,4.208955,67,193,4
4,2X5G4Ujq0s4Wfn4TC7gX0g,9Ify25DK87s5_u2EhK0_Rg,4.5,56,4.300000,10,193,4
5,2X5G4Ujq0s4Wfn4TC7gX0g,WzaaorVCmUTQvu4mScunNg,4.5,137,4.333333,24,193,4
6,2X5G4Ujq0s4Wfn4TC7gX0g,neBCZl1X_pDl11y7nVYFoQ,4.5,5,3.000000,1,193,4
7,2X5G4Ujq0s4Wfn4TC7gX0g,9OZH1Ecw-qUkCW5MS0NefA,4.5,1138,4.230769,52,193,4
8,2X5G4Ujq0s4Wfn4TC7gX0g,2Az24juAhTJtmlqr5UWXSQ,4.5,17,4.500000,6,193,5
9,2X5G4Ujq0s4Wfn4TC7gX0g,WMTm9HHRA3EewoxTX1Gleg,4.5,59,4.214286,14,193,5


In [69]:
#this is the one making the most sense
plt.axhline(y=4.5)
plt.scatter(final_df_4_point_5.index, final_df_4_point_5.user_rating, c='b', label='user ratings')
plt.scatter(final_df_4_point_5.index, final_df_4_point_5.friends_avg_rating, c='r', label='friends_avg')

In [70]:
plt.show()